In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

device = "cpu"
import pandas as pd
from bs4 import BeautifulSoup
import itertools
import more_itertools
import numpy as np
import pickle
import json

In [2]:
device

'cpu'

In [3]:
path = "dataset/"
df = pd.read_csv(path+"USvideos.csv")
df = df.assign(country="US")
# df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m')  
# df.trending_date = df.trending_date.dt.date   
# df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
# df=df.assign(publish_date=df['publish_time'].dt.date)
# df['publish_time'] = df['publish_time'].dt.time

# ###导入category名称###
# df=df.assign(cat_name='a')
# id_to_category = {}
# file=path+'US_category_id.json'
# with open(file, 'r') as f:
#     data=json.load(f)
#     for category in data['items']:
#         id_to_category[category['id']] = category['snippet']['title']
# print(id_to_category)
# ###实际上每个国家的category id-name 字典是一样的
# df['category_id'] = df['category_id'].astype(str)
# df.insert(4, 'category', df['category_id'].map(id_to_category))

C:\Users\gx\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,description,views,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,country
0,SHANTELL'S CHANNEL - https://www.youtube.com/s...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
1,"One year after the presidential election, John...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
2,WATCH MY PREVIOUS VIDEO 鈻?\n\nSUBSCRIBE 鈻?http...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
3,Today we find out if Link is a Nickelback amat...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
4,I know it's been a while since we did this sho...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US


In [5]:

## mark the columns which contains text for classification and target class
col_text = 'description'
col_target = 'views'

In [6]:
cls_arr = np.sort(df[col_target].unique()).tolist()
classes = len(cls_arr)
classes

3

In [7]:
## divide dataset in 80% train 10% validation 10% test as done in the paper
length = df.shape[0]
train_len = int(0.8*length)
val_len = int(0.1*length)

train = df[:train_len]
val = df[train_len:train_len+val_len]
test = df[train_len+val_len:]

In [8]:
def clean_str(string, max_seq_len):
    """
    adapted from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = str(string)
    string = BeautifulSoup(string, "lxml").text
    string = re.sub(r"[^A-Za-z0-9(),!?\"\`]", " ", string)
    string = re.sub(r"\"s", " \"s", string)
    string = re.sub(r"\"ve", " \"ve", string)
    string = re.sub(r"n\"t", " n\"t", string)
    string = re.sub(r"\"re", " \"re", string)
    string = re.sub(r"\"d", " \"d", string)
    string = re.sub(r"\"ll", " \"ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    s =string.strip().lower().split(" ")
    if len(s) > max_seq_len:
        return s[0:max_seq_len] 
    return s

In [9]:
## creates a 3D list of format paragraph[sentence[word]]
def create3DList(df,col, max_sent_len,max_seq_len):
    x=[]
    for docs in df[col].as_matrix():
        x1=[]
        idx = 0
        docs = str(docs)
        for seq in "|||".join(re.split("[.?!]", docs)).split("|||"):
            x1.append(clean_str(seq,max_sent_len))
            if(idx>=max_seq_len-1):
                break
            idx= idx+1
        x.append(x1)
    return x

In [10]:
## Fix the maximum length of sentences in a paragraph and words in a sentence
max_sent_len = 12
max_seq_len = 25
# print(df[col].as_matrix()[0])

In [11]:
## divides review in sentences and sentences into word creating a 3DList
x_train = create3DList(train,col_text, max_sent_len,max_seq_len)
x_val = create3DList(val, col_text, max_sent_len,max_seq_len)
x_test = create3DList(test, col_text, max_sent_len,max_seq_len)
print("x_train: {}".format(len(x_train)))
print("x_val: {}".format(len(x_val)))
print("x_test: {}".format(len(x_test)))

C:\Users\gx\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://youtu" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://www" looks like a URL. Beautiful Soup is not an HTTP client.

C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://sub2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://snapchat" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://Teespring" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C

C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://discuss" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://represent" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://carlyanderin" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decode

C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://damonandjo" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://kinded" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://annaed" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_marku

C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://eepurl" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://intuit" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://apple" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\U

C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://m" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://beaklondike" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\gx\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://skl" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Use

x_train: 32580
x_val: 4072
x_test: 4074


In [12]:

from gensim.models import Word2Vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import string


In [13]:
stoplist = stopwords.words('english') + list(string.punctuation)
stemmer = SnowballStemmer('english')
x_train_texts = [[[stemmer.stem(word.lower()) for word in sent  if word not in stoplist] for sent in para]
         for para in x_train]
x_test_texts = [[[stemmer.stem(word.lower()) for word in sent  if word not in stoplist] for sent in para]
         for para in x_test]
x_val_texts = [[[stemmer.stem(word.lower()) for word in sent  if word not in stoplist] for sent in para]
         for para in x_val]

## calculate frequency of words
from collections import defaultdict
frequency1 = defaultdict(int)
for texts in x_train_texts:     
    for text in texts:
        for token in text:
            frequency1[token] += 1
for texts in x_test_texts:     
    for text in texts:
        for token in text:
            frequency1[token] += 1
for texts in x_val_texts:     
    for text in texts:
        for token in text:
            frequency1[token] += 1
            
## remove  words with frequency less than 5.
x_train_texts = [[[token for token in text if frequency1[token] > 5]
         for text in texts] for texts in x_train_texts]

x_test_texts = [[[token for token in text if frequency1[token] > 5]
         for text in texts] for texts in x_test_texts]
x_val_texts = [[[token for token in text if frequency1[token] > 5]
         for text in texts] for texts in x_val_texts]

In [14]:
texts = list(more_itertools.collapse(x_train_texts[:] + x_test_texts[:] + x_val_texts[:],levels=1))


In [15]:
word2vec = Word2Vec(texts,size=200, min_count=5)


In [16]:
word2vec.save("dictonary_youtube")


In [17]:
## convert 3D text list to 3D list of index 
x_train_vec = [[[word2vec.wv.vocab[token].index for token in text]
         for text in texts] for texts in x_train_texts]

In [18]:
x_test_vec = [[[word2vec.wv.vocab[token].index for token in text]
         for text in texts] for texts in x_test_texts]

In [19]:
x_val_vec = [[[word2vec.wv.vocab[token].index for token in text]
         for text in texts] for texts in x_val_texts]

In [20]:
weights = torch.FloatTensor(word2vec.wv.syn0)

C:\Users\gx\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [21]:
vocab_size = len(word2vec.wv.vocab)

In [22]:
y_train = train[col_target].tolist()
y_test = test[col_target].tolist()
y_val = val[col_target].tolist()

In [23]:
## Make the the multiple attention with word vectors.
def attention_mul(rnn_outputs, att_weights):
    attn_vectors = None
    for i in range(rnn_outputs.size(0)):
        h_i = rnn_outputs[i]
        a_i = att_weights[i]
        h_i = a_i * h_i
        h_i = h_i.unsqueeze(0)
        if(attn_vectors is None):
            attn_vectors = h_i
        else:
            attn_vectors = torch.cat((attn_vectors,h_i),0)
    return torch.sum(attn_vectors, 0).unsqueeze(0)

In [24]:
## The word RNN model for generating a sentence vector
class WordRNN(nn.Module):
    def __init__(self, vocab_size,embedsize, batch_size, hid_size):
        super(WordRNN, self).__init__()
        self.batch_size = batch_size
        self.embedsize = embedsize
        self.hid_size = hid_size
        ## Word Encoder
        self.embed = nn.Embedding(vocab_size, embedsize)
        self.wordRNN = nn.GRU(embedsize, hid_size, bidirectional=True)
        ## Word Attention
        self.wordattn = nn.Linear(2*hid_size, 2*hid_size)
        self.attn_combine = nn.Linear(2*hid_size, 2*hid_size,bias=False)
    def forward(self,inp, hid_state):
        emb_out  = self.embed(inp)

        out_state, hid_state = self.wordRNN(emb_out, hid_state)

        word_annotation = self.wordattn(out_state)
        attn = F.softmax(self.attn_combine(word_annotation),dim=1)

        sent = attention_mul(out_state,attn)
        return sent, hid_state

In [25]:
## The HAN model
class SentenceRNN(nn.Module):
    def __init__(self,vocab_size,embedsize, batch_size, hid_size,c):
        super(SentenceRNN, self).__init__()
        self.batch_size = batch_size
        self.embedsize = embedsize
        self.hid_size = hid_size
        self.cls = c
        self.wordRNN = WordRNN(vocab_size,embedsize, batch_size, hid_size)
        ## Sentence Encoder
        self.sentRNN = nn.GRU(embedsize, hid_size, bidirectional=True)
        ## Sentence Attention
        self.sentattn = nn.Linear(2*hid_size, 2*hid_size)
        self.attn_combine = nn.Linear(2*hid_size, 2*hid_size,bias=False)
        self.doc_linear = nn.Linear(2*hid_size, c)
    
    def forward(self,inp, hid_state_sent, hid_state_word):
        s = None
        ## Generating sentence vector through WordRNN
        for i in range(len(inp[0])):
            r = None
            for j in range(len(inp)):
                if(r is None):
                    r = [inp[j][i]]
                else:
                    r.append(inp[j][i])
            r1 = np.asarray([sub_list + [0] * (max_seq_len - len(sub_list)) for sub_list in r])
            _s, state_word = self.wordRNN(torch.LongTensor(r1).view(-1,batch_size), hid_state_word)
            if(s is None):
                s = _s
            else:
                s = torch.cat((s,_s),0)

                out_state, hid_state = self.sentRNN(s, hid_state_sent)
        sent_annotation = self.sentattn(out_state)
        attn = F.softmax(self.attn_combine(sent_annotation),dim=1)

        doc = attention_mul(out_state,attn)
        d = self.doc_linear(doc)
        cls = F.log_softmax(d.view(-1,self.cls),dim=1)
        return cls, hid_state
    
    def init_hidden_sent(self):
            return Variable(torch.zeros(2, self.batch_size, self.hid_size))
    
    def init_hidden_word(self):
            return Variable(torch.zeros(2, self.batch_size, self.hid_size))

In [26]:
# ## converting list to tensor
# # y_train[:10]
y_train_tensor =  torch.from_numpy(np.array(y_train))#[torch.FloatTensor([cls_arr[label]]) for label in y_train]
y_val_tensor =  torch.from_numpy(np.array(y_val))#[torch.FloatTensor([cls_arr[label]]) for label in y_val]
y_test_tensor =  torch.from_numpy(np.array(y_test))#[torch.FloatTensor([cls_arr[label]]) for label in y_test]


# ## converting list to tensor
# y_train_tensor =  [torch.FloatTensor([cls_arr.index(label)]) for label in y_train]
# y_val_tensor =  [torch.FloatTensor([cls_arr.index(label)]) for label in y_val]
# y_test_tensor =  [torch.FloatTensor([cls_arr.index(label)]) for label in y_test]

In [27]:
max_seq_len = max([len(seq) for seq in itertools.chain.from_iterable(x_train_vec +x_val_vec + x_test_vec)])
max_sent_len = max([len(sent) for sent in (x_train_vec + x_val_vec + x_test_vec)])

In [28]:
max_seq_len

12

In [29]:

max_sent_len

25

In [30]:
np.percentile(np.array([len(seq) for seq in itertools.chain.from_iterable(x_train_vec +x_val_vec + x_test_vec)]),90)

9.0

In [31]:
np.percentile(np.array([len(sent) for sent in (x_train_vec +x_val_vec + x_test_vec)]),90)


25.0

In [32]:
## Padding the input 
X_train_pad = [sub_list + [[0]] * (max_sent_len - len(sub_list)) for sub_list in x_train_vec]
X_val_pad = [sub_list + [[0]] * (max_sent_len - len(sub_list)) for sub_list in x_val_vec]
X_test_pad = [sub_list + [[0]] * (max_sent_len - len(sub_list)) for sub_list in x_test_vec]

In [33]:
batch_size = 64

In [34]:
def train_data(batch_size, review, targets, sent_attn_model, sent_optimizer, criterion):

    state_word = sent_attn_model.init_hidden_word()
    state_sent = sent_attn_model.init_hidden_sent()
    sent_optimizer.zero_grad()
            
    y_pred, state_sent = sent_attn_model(review, state_sent, state_word)

    loss = criterion(y_pred, torch.LongTensor(targets)) 
    
    max_index = y_pred.max(dim = 1)[1]
    correct = (max_index == torch.LongTensor(targets)).sum()
    acc = float(correct)/batch_size

    loss.backward()
#     print(loss)
    sent_optimizer.step()
    
    return loss.item(),acc

In [35]:
hid_size = 100
embedsize = 200

In [36]:
sent_attn = SentenceRNN(vocab_size,embedsize,batch_size,hid_size,classes)
sent_attn.wordRNN.embed.from_pretrained(weights)
torch.backends.cudnn.benchmark=True

In [37]:
learning_rate = 1e-2
momentum = 0.9

sent_optimizer = torch.optim.SGD(sent_attn.parameters(), lr=learning_rate, momentum= momentum)

criterion = nn.NLLLoss()

In [38]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [39]:
def gen_batch(x,y,batch_size):
    k = random.sample(range(len(x)-1),batch_size)
    x_batch=[]
    y_batch=[]

    for t in k:
        x_batch.append(x[t])
        y_batch.append(y[t])

    return [x_batch,y_batch]

In [40]:
def validation_accuracy(batch_size, x_val,y_val,sent_attn_model):
    acc = []
    val_length = len(x_val)
    for j in range(int(val_length/batch_size)):
        x,y = gen_batch(x_val,y_val,batch_size)
        state_word = sent_attn_model.init_hidden_word()
        state_sent = sent_attn_model.init_hidden_sent()
        
        y_pred, state_sent = sent_attn_model(x, state_sent, state_word)
        max_index = y_pred.max(dim = 1)[1]
        correct = (max_index == torch.LongTensor(y)).sum()
        acc.append(float(correct)/batch_size)
    return np.mean(acc)

In [41]:
def train_early_stopping(batch_size, x_train, y_train, x_val, y_val, sent_attn_model, 
                         sent_attn_optimiser, loss_criterion, num_epoch,
                         print_loss_every = 50, code_test=True):
    start = time.time()
    loss_full = []
    loss_epoch = []
    acc_epoch = []
    acc_full = []
    val_acc = []
    epoch_counter = 0
    train_length = len(x_train)
    for i in range(1, num_epoch + 1):
        loss_epoch = []
        acc_epoch = []
        for j in range(int(train_length/batch_size)):
            x,y = gen_batch(x_train,y_train,batch_size)
            loss,acc = train_data(batch_size, x, y, sent_attn_model, sent_attn_optimiser, loss_criterion)
            loss_epoch.append(loss)
            acc_epoch.append(acc)
            if (code_test and j % int(print_loss_every/batch_size) == 0) :
                print ('Loss at %d paragraphs, %d epoch,(%s) is %f' %(j*batch_size, i, timeSince(start), np.mean(loss_epoch)))
                print ('Accuracy at %d paragraphs, %d epoch,(%s) is %f' %(j*batch_size, i, timeSince(start), np.mean(acc_epoch)))
        
        loss_full.append(np.mean(loss_epoch))
        acc_full.append(np.mean(acc_epoch))
        torch.save(sent_attn_model.state_dict(), 'sent_attn_model_yelp.pth')
        print ('Loss after %d epoch,(%s) is %f' %(i, timeSince(start), np.mean(loss_epoch)))
        print ('Train Accuracy after %d epoch,(%s) is %f' %(i, timeSince(start), np.mean(acc_epoch)))

        val_acc.append(validation_accuracy(batch_size, x_val, y_val, sent_attn_model)) 
        print ('Validation Accuracy after %d epoch,(%s) is %f' %(i, timeSince(start), val_acc[-1]))
    return loss_full,acc_full,val_acc

In [42]:
epoch = 200

In [ ]:
loss_full, acc_full, val_acc = train_early_stopping(batch_size, X_train_pad, y_train_tensor, X_val_pad,
                                y_val_tensor, sent_attn, sent_optimizer, criterion, epoch, 10000, False)


Loss after 1 epoch,(22m 20s) is 0.954827
Train Accuracy after 1 epoch,(22m 20s) is 0.579261
Validation Accuracy after 1 epoch,(23m 34s) is 0.285962
Loss after 2 epoch,(45m 46s) is 0.954554
Train Accuracy after 2 epoch,(45m 46s) is 0.577879
Validation Accuracy after 2 epoch,(47m 0s) is 0.286458
Loss after 3 epoch,(69m 3s) is 0.950562
Train Accuracy after 3 epoch,(69m 3s) is 0.582944
Validation Accuracy after 3 epoch,(70m 16s) is 0.285218
Loss after 4 epoch,(92m 29s) is 0.954126
Train Accuracy after 4 epoch,(92m 29s) is 0.579568
Validation Accuracy after 4 epoch,(93m 43s) is 0.280258
Loss after 5 epoch,(115m 47s) is 0.957181
Train Accuracy after 5 epoch,(115m 47s) is 0.579107
Validation Accuracy after 5 epoch,(117m 0s) is 0.286210
Loss after 6 epoch,(139m 12s) is 0.961063
Train Accuracy after 6 epoch,(139m 12s) is 0.573735
Validation Accuracy after 6 epoch,(140m 27s) is 0.295387
Loss after 7 epoch,(162m 37s) is 0.956645
Train Accuracy after 7 epoch,(162m 37s) is 0.577020
Validation Accur

Train Accuracy after 54 epoch,(1419m 46s) is 0.577726
Validation Accuracy after 54 epoch,(1420m 43s) is 0.287698
Loss after 55 epoch,(1438m 26s) is 0.952133
Train Accuracy after 55 epoch,(1438m 26s) is 0.581717
Validation Accuracy after 55 epoch,(1439m 23s) is 0.277530
Loss after 56 epoch,(1457m 4s) is 0.959518
Train Accuracy after 56 epoch,(1457m 4s) is 0.573520
Validation Accuracy after 56 epoch,(1458m 3s) is 0.274802
Loss after 57 epoch,(1475m 46s) is 0.955042
Train Accuracy after 57 epoch,(1475m 46s) is 0.579752
Validation Accuracy after 57 epoch,(1476m 42s) is 0.269841
Loss after 58 epoch,(1494m 1s) is 0.951682
Train Accuracy after 58 epoch,(1494m 1s) is 0.582852
Validation Accuracy after 58 epoch,(1494m 56s) is 0.289931
